In [55]:
import pandas as pd
import math

In [56]:
def entropy(values):
    result = 0.0
    div = sum(values)
    for v in values:
        if v > 0:
            result -= v/div * math.log(v/div, 2)
    return result

In [57]:
def entropyTotale(values):
    result = 0.0
    total = 0
    for e in values:
        total += sum(e)
    for e in values:
        card = sum(e)
        result += entropy(e) * card / total
    return result

In [58]:
def entropyContinu(values):
    result = 0
    total = 0
    for e in values:
        total += sum(e)
    for e in values:
        print("****", entropy(e) * sum(e) / total)
        result += entropy(e) * sum(e) / total
    return result

In [59]:
import numpy as np


def frequency_by_bins(df: pd.DataFrame, first_col: str, second_col: str):
    # Compute the bin edges as the average of each pair of consecutive values in the first column
    bin_edges = (df[first_col] + df[first_col].shift()) / 2

    # Drop the first row since it will have a NaN value for the bin edge
    bin_edges = bin_edges.dropna()

    # Initialize empty dictionaries to store the frequencies before and after each bin edge
    freqs_before = {}
    freqs_after = {}

    min_entropy = 1000
    min_avg = 0
    min_info = []
    # Iterate over the bin edges and compute the frequencies before and after each one
    for edge in bin_edges:
        glob_list = []
        sub_list1 = []
        sun_list2 = []
        print(edge)
        # Find the index of the current bin edge in the original DataFrame
        idx = df[first_col].searchsorted(edge)
        #print("Before ==> ")
        for v in df.iloc[:idx][second_col].value_counts():
            #print("  ", v)
            sub_list1.append(v)
        glob_list.append(sub_list1)
        #print("After ==> ")
        for v in df.iloc[idx:][second_col].value_counts():
            #print("  ", v)
            sun_list2.append(v)
        glob_list.append(sun_list2)
        print(glob_list)

        entr_cont = entropyContinu(glob_list)
        print("entropy =====> ", entr_cont)
        if entr_cont < min_entropy:
            min_entropy = entr_cont
            min_avg = edge

        # Compute the frequencies of the second column before and after the current bin edge
        freqs_before[edge] = df.iloc[:idx][second_col].value_counts()
        freqs_after[edge] = df.iloc[idx:][second_col].value_counts()

        # Combine the dictionaries into a single DataFrame and return it
    #freqs_df = pd.concat(
        #[pd.concat([freqs_before[k], freqs_after[k]], axis=1, keys=['before', 'after'])
         #for k in freqs_before.keys()],
        #axis=1
    #)
    #freqs_df.columns.names = ['bin', 'value']
    print(min_avg)
    print(min_entropy)
    min_info.append(min_entropy)
    min_info.append(min_avg)
    min_info.append(first_col)

    #df1 = df[df[first_col] < 57.5]
    #df2 = df[df[first_col] >= 57.5]

    # Print the resulting groups
    #print('Group 1:\n', df1)
    #print('Group 2:\n', df2)

    return min_info

    #print(freqs_df)

    #return groups

In [60]:
#data = pd.read_csv('data.csv')
#frequency_by_bins(data, 'Poids', 'class')

In [61]:
def group_by_column(df: pd.DataFrame, column_to_group: str):
    groups = df.drop(column_to_group, axis=1).groupby(df[column_to_group])
    for group_name, group_data in groups:
        print("\n*Group:", group_name)
        print(group_data)
        id3(group_data)

In [62]:
def id3(df: pd.DataFrame, test_date: pd.DataFrame = None, ignore_cols:list=[]):
    ignore_cols.append('class')
    ignore_cols.append('id')
    class_col = 'class'
    numeric_cols = df.select_dtypes(include='number').columns
    numeric_cols = [col for col in numeric_cols if col not in ignore_cols]

    cont_attr = []
    for col in numeric_cols:
        #counts = df.groupby(class_col)[col].value_counts()
        #print(f"Value counts for column '{col}' by {class_col}:\n{counts}\n")
        print(f"\nNumeric column '{col}':")
        cont_attr = frequency_by_bins(df, col, class_col)
    if not cont_attr:
        cont_attr = [1000]

    non_numeric_cols = df.select_dtypes(include='object').columns.tolist()
    non_numeric_cols = [col for col in non_numeric_cols if col not in ignore_cols]

    non_numeric_counts = {}
    for col in non_numeric_cols:
        counts = df.groupby(class_col)[col].value_counts()
        non_numeric_counts[col] = counts
        #print(f"Value counts for column '{col}' by {class_col}:\n{counts}\n")

    non_numeric_entropys = {}
    for col, counts in non_numeric_counts.items():
        print(f"\nNon-numeric column '{col}':")
        entropy_list = []
        for col_value in df[col].unique():
            group_counts = df.groupby(col)[class_col].value_counts()
            #print("*****", group_counts)
            try:
                attr_list = []
                #print("**********", col_value)
                value_counts = group_counts[col_value]
                for v in value_counts.to_dict().values():
                    #print(v)
                    attr_list.append(v)
                entropy_list.append(attr_list)
                print(f"  '{col_value} ==> '{entropy(attr_list)}'")
                #print(attr_list)
                #print(f"Value '{col_value}': {value_counts.to_dict()}")
            except KeyError:
                print(f"Value '{col_value}': {0}")
        #print(entropy_list)
        non_numeric_entropys[col] = entropyTotale(entropy_list)
        print(f"column'{col}' ==> entropy : '{non_numeric_entropys[col]}'")
    #print(min(non_numeric_entropys, key=non_numeric_entropys.get))
    #print(min(non_numeric_entropys.values()))
    min_non_numeric = 1000
    if sum(non_numeric_entropys.values()) == 0.0 and (cont_attr[0] == 0.0 or cont_attr[0] == 1000):
        print("******No more split")
    else:
        if sum(non_numeric_entropys.values()) > 0.0:
            min_non_numeric = min(non_numeric_entropys.values())
        if min_non_numeric < cont_attr[0]:
            #print(cont_attr[0])
            print("===========> split by : ", min(non_numeric_entropys, key=non_numeric_entropys.get))
            group_by_column(df, min(non_numeric_entropys, key=non_numeric_entropys.get))
        else:
            print("===========> split by : ", cont_attr[2])
            df1 = df[df[cont_attr[2]] < cont_attr[1]]
            df2 = df[df[cont_attr[2]] >= cont_attr[1]]
            print("\n*Group < ", cont_attr[1])
            print(df1)
            id3(df1)
            print("\n*Group >= ", cont_attr[1])
            print(df2)
            id3(df2)
        #group_by_column(df, min(non_numeric_entropys, key=non_numeric_entropys.get))


In [63]:
data = pd.read_csv('data.csv')
id3(data)


Numeric column 'Poids':
42.5
[[1], [4, 3]]
**** 0.0
**** 0.8620746190299702
entropy =====>  0.8620746190299702
50.0
[[2], [4, 2]]
**** 0.0
**** 0.6887218755408672
entropy =====>  0.6887218755408672
57.5
[[3], [4, 1]]
**** 0.0
**** 0.4512050593046014
entropy =====>  0.4512050593046014
62.5
[[4], [4]]
**** 0.0
**** 0.0
entropy =====>  0.0
70.0
[[4, 1], [3]]
**** 0.4512050593046014
**** 0.0
entropy =====>  0.4512050593046014
77.5
[[4, 2], [2]]
**** 0.6887218755408672
**** 0.0
entropy =====>  0.6887218755408672
82.5
[[4, 3], [1]]
**** 0.8620746190299702
**** 0.0
entropy =====>  0.8620746190299702
62.5
0.0

Non-numeric column 'Cheveux':
  'bl ==> '0.8112781244591328'
  'br ==> '0.9182958340544896'
  'r ==> '0.0'
column'Cheveux' ==> entropy : '0.75'

Non-numeric column 'Taille':
  'p ==> '0.9182958340544896'
  'm ==> '0.9182958340544896'
  'g ==> '1.0'
column'Taille' ==> entropy : '0.9387218755408672'

Non-numeric column 'Vegetarien':
  'o ==> '0.0'
  'n ==> '0.7219280948873623'
column'Vege